In [ ]:
#| default_exp prepare_t2s_txts

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import sys
import os
import itertools
from pathlib import Path

import numpy as np
import torch
import torch.nn.functional as F

from fastprogress import progress_bar
from fastcore.script import *

import whisper, whisperx
from whisperspeech import utils, vad_merge
import webdataset as wds

from whisperspeech.inference import get_compute_device

/opt/conda/lib/python3.10/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/opt/conda/lib/python3.10/site-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


# T2S dataset preparation

We take a webdataset shard and extract semantic tokens and transcriptions from it.

We use VAD chunks merged with randomized maximum length to also generate some short samples.

In [ ]:
#| exporti
class Transcriber:
    """
    A helper class to transcribe a batch of 30 second audio chunks.
    """
    def __init__(self, model_size, lang=False):
        self.model = whisperx.asr.load_model(
            model_size, get_compute_device(), compute_type="float16", language=lang,
            asr_options=dict(repetition_penalty=1, no_repeat_ngram_size=0, prompt_reset_on_temperature=0.5,
                             max_new_tokens=500, clip_timestamps=None, hallucination_silence_threshold=None))
        # without calling vad_model at least once the rest segfaults for some reason...
        self.model.vad_model({"waveform": torch.zeros(1, 16000), "sample_rate": 16000})
        
    def transcribe(self, batch):
        batch = whisper.log_mel_spectrogram(batch)
        embs = self.model.model.encode(batch.cpu().numpy())
        return self.model.tokenizer.tokenizer.decode_batch([x.sequences_ids[0] for x in 
            self.model.model.model.generate(
                embs,
                [self.model.model.get_prompt(self.model.tokenizer, [], without_timestamps=True)]*len(batch),
            )])

In [ ]:
#| exporti
@call_parse
def prepare_txt(
    input:str,  # FLAC webdataset file path (or - to read the names from stdin)
    n_samples:int=None, # process a limited amount of samples
    batch_size:int=1, # process several segments at once
    transcription_model:str="small.en",
    language:str=False,
    skip_first_and_last:bool=False,
):
    transcriber = Transcriber(transcription_model, lang=language)
#     whmodel = whisper.load_model(transcription_model)
#     decoding_options = whisper.DecodingOptions(language=language)
#     for b in whmodel.decoder.blocks:
#         b.attn.qkv_attention = b.attn.qkv_attention_old

    total = n_samples//batch_size if n_samples else 'noinfer'
    if n_samples: print(f"Benchmarking run of {n_samples} samples ({total} batches)")

    ds = vad_merge.chunked_audio_dataset([input], 'eqvad').compose(
        utils.resampler(16000, 'samples_16k'),
    )
    
    if skip_first_and_last:
        # when processing LibriLight we drop the first and last segment because they tend
        # to be inaccurate (the transcriptions lack the "LibriVox ad" prefixes and
        # "end of chapter" suffixes)
        ds = ds.compose(
            wds.select(lambda x: x['i'] != 0 and x['i'] != x['imax']),
        )
    
    ds = ds.compose(
        wds.to_tuple('__key__', 'rpad', 'samples_16k'),
        wds.batched(64),
    )

    dl = wds.WebLoader(ds, num_workers=1, batch_size=None).unbatched().batched(batch_size)

    with utils.AtomicTarWriter(utils.derived_name(input, f'{transcription_model}-txt', dir="."), throwaway=n_samples is not None) as sink:
        for keys, rpads, samples in progress_bar(dl, total=total):
            csamples = samples.to(get_compute_device())
            txts = transcriber.transcribe(csamples)
            # with torch.no_grad():
            #     embs = whmodel.encoder(whisper.log_mel_spectrogram(csamples))
            #     decs = whmodel.decode(embs, decoding_options)
            #     txts = [x.text for x in decs]

            for key, rpad, txt in zip(keys, rpads, txts):
                sink.write({
                    "__key__": key,
                    "txt": txt,
                })

In [ ]:
prepare_txt('../wolnelektury-wds2/wolnelektury-audio-000000.tar', n_samples=1024, batch_size=16)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.1.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.0+cu121. Bad things might happen unless you revert torch to 1.x.
Benchmarking run of 1024 samples (64 batches)


In [ ]:
prepare_txt('../wolnelektury-wds2/wolnelektury-audio-000000.tar', n_samples=1024, batch_size=16)

Benchmarking run of 1024 samples (64 batches)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
prepare_txt('../wolnelektury-wds2/wolnelektury-audio-000000.tar', transcription_model='medium', n_samples=1024, batch_size=16)

Benchmarking run of 1024 samples (64 batches)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
prepare_txt('../wolnelektury-wds2/wolnelektury-audio-000000.tar', transcription_model='medium', n_samples=1024, batch_size=1)

Benchmarking run of 1024 samples (1024 batches)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()